In [1]:
import mlrun
from mlrun import  get_or_create_project
project_name = "feature-sets"
project = get_or_create_project(project_name, context="./")

> 2025-03-12 17:18:15,535 [info] Project loaded successfully: {"project_name":"feature-sets"}


In [2]:
import mlrun.feature_store as fs
from mlrun.feature_store.steps import *
from mlrun.features import MinMaxValidator
from mlrun.datastore.targets import ParquetTarget, NoSqlTarget,CSVTarget

In [3]:
import pandas as pd
quotes = pd.DataFrame(
    {
        "time": [
            pd.Timestamp("2016-05-25 13:30:00.023"),
            pd.Timestamp("2016-05-25 13:30:00.023"),
            pd.Timestamp("2016-05-25 13:30:00.030"),
            pd.Timestamp("2016-05-25 13:30:00.041"),
            pd.Timestamp("2016-05-25 13:30:00.048"),
            pd.Timestamp("2016-05-25 13:30:00.049"),
            pd.Timestamp("2016-05-25 13:30:00.072"),
            pd.Timestamp("2016-05-25 13:30:00.075")
        ],
        "ticker": [
               "GOOG",
               "MSFT",
               "MSFT",
               "MSFT",
               "GOOG",
               "AAPL",
               "GOOG",
               "MSFT"
           ],
           "bid": [720.50, 51.95, 51.97, 51.99, 720.50, 97.99, 720.50, 52.01],
           "ask": [720.93, 51.96, 51.98, 52.00, 720.93, 98.01, 720.88, 52.03],
    }
)

trades = pd.DataFrame(
       {
           "time": [
               pd.Timestamp("2016-05-25 13:30:00.023"),
               pd.Timestamp("2016-05-25 13:30:00.038"),
               pd.Timestamp("2016-05-25 13:30:00.048"),
               pd.Timestamp("2016-05-25 13:30:00.048"),
               pd.Timestamp("2016-05-25 13:30:00.048")
           ],
           "ticker": ["MSFT", "MSFT", "GOOG", "GOOG", "AAPL"],
           "price": [51.95, 51.95, 720.77, 720.92, 98.0],
           "quantity": [75, 155, 100, 100, 100]
       }
)


stocks = pd.DataFrame(
       {
           "ticker": ["MSFT", "GOOG", "AAPL"],
           "name": ["Microsoft Corporation", "Alphabet Inc", "Apple Inc"],
           "exchange": ["NASDAQ", "NASDAQ", "NASDAQ"]
       }
)


In [4]:
import datetime
def move_date(df, col):
    max_date = df[col].max()
    now_date = datetime.datetime.now()
    delta = now_date - max_date 
    df[col] = df[col] + delta 
    return df

In [5]:
quotes = move_date(quotes, "time")
trades = move_date(trades, "time")

## Creating fs with 5 tags

In [7]:
import mlrun.feature_store as fs
from mlrun.feature_store import FeatureSet, Entity
for i in range(5):
    quotes_set = fs.FeatureSet("quotes", "my_quotes_description1234567890",entities=[fs.Entity("ticker")], timestamp_key= "time")
    quotes_set.metadata.labels = {'label-key1': 'label-value1','label-key2': 'label-value2','label-key3': 'label-value3','label-key4': 'label-value4','label-key5': 'label-value5','label-key6': 'label-value6','label-key7': 'label-value7','label-key7': 'label-value7','label-key9': 'label-value9','label-key10': 'label-value10'}
    #quotes_set.set_targets()
    partition_cols = ['ask', '$key']
    targets = [ParquetTarget(partitioned=True, partition_cols=partition_cols), NoSqlTarget(), CSVTarget() ]
    quotes_set.spec.label_column= 'ticker'
    quotes_set.spec.partition_keys = 'ticker'
    quotes_set.spec.relations={'key':'value','key2':'value2'}
    quotes_set["bid"] = fs.Feature(description= "desc_bid",labels= {'bid-key1': 'bid-value1'}, validator=MinMaxValidator(min=55, severity="info"))
    quotes_set.save(tag='tag'+str(i), versioned=True)
    ingest_df = quotes_set.ingest(quotes, targets, infer_options=fs.InferOptions.default())

## Creating fv

In [8]:
features = [
    "quotes.*"
]

vector = fs.FeatureVector("vector-name",features=features,description="this is my vector")
vector.metadata.labels = {'label-key1': 'label-value1','label-key2': 'label-value2','label-key3': 'label-value3',}
svc = vector.get_online_feature_service()
resp = svc.get([{"ticker": "MSFT"}, {"ticker": "AAPL"}])
try:
    resp1 = vector.get_offline_features()
except:
    print("Error occurred")

Error occurred
